# TP1
## Grupo 17:
**PG50315 - David Alexandre Ferreira Duarte**

**PG51247 - João Rafael Cerqueira Monteiro**

## Exercício 1. 
Use a package Criptography  para 
Criar um comunicação privada assíncrona entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:
    1. Autenticação do criptograma e dos metadados (associated data). Usar uma cifra simétrica  num modo **HMAC**  que seja seguro contra ataques aos “nounces” .
    2. Os “nounces” são gerados por um gerador pseudo aleatório (PRG) construído por um função de hash em modo XOF.
    3. O par de chaves $$cipher\_key$$, $$mac\_key$$ , para cifra e autenticação, é acordado entre agentes usando o protocolo ECDH com autenticação dos agentes usando assinaturas ECDSA.

In [1]:
!pip install cryptography
!pip install nest_asyncio

In [2]:
# Import
import hashlib
import hmac
import os
import asyncio
import json
from typing import Tuple
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend


In [3]:
class funcoesCifragem:
    def __init__(self):
        self.algoritmoHash = hashlib.sha256
        self.algoritmoCifra = algorithms.AES
        self.modoCifra = modes.CTR
        self.algoritmoHMAC = hashlib.sha256
        self.digest_size = self.algoritmoHMAC().digest_size

    def gerarParChaves(self):
        chavePrivada = ec.generate_private_key(ec.SECP256R1())
        chavePublica = chavePrivada.public_key()
        return chavePrivada, chavePublica

    def serializarChavePublica(self, chavePublica):
        return chavePublica.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )

    def deserializarChavePublica(self, bytesChavePublica):
        return serialization.load_pem_public_key(
            bytesChavePublica,
            backend=default_backend()
        )

    def gerarNounce(self, size=17):
        return os.urandom(size)

    def derivadaHKDF(self, segredo, salt, info=b'', size=None):
        hkdf = HKDF(
            algorithm=self.algoritmoHash(),
            length=size or self.digest_size,
            salt=salt,
            info=info,
            backend=default_backend()
        )
        return hkdf.derive(segredo)

    def cifrar(self, plain_text, cipher_key, nonce):
        cifra = Cipher(self.algoritmoCifra(cipher_key), self.modoCifra(nonce))
        encryptor = cifra.encryptor()
        textoCifrado = encryptor.update(plain_text) + encryptor.finalize()
        return textoCifrado

    def decifrar(self, textoCifrado, cipher_key, nonce):
        cifra = Cipher(self.algoritmoCifra(cipher_key), self.modoCifra(nonce))
        decryptor = cifra.decryptor()
        plain_text = decryptor.update(textoCifrado) + decryptor.finalize()
        return plain_text

    def verificarHMAC(self, message, mac_key, digest):
        h = hmac.new(mac_key, message, self.algoritmoHMAC)
        return hmac.compare_digest(h.digest(), digest)

In [4]:
class Emitter:
    def __init__(self, receiverBytesChavePublica):
        self.crypto = funcoesCifragem()
        self.receiverChavePublica = self.crypto.deserializarChavePublica(receiverBytesChavePublica)
        self.chavePrivada, self.chavePublica = self.crypto.gerarParChaves()
        self.cipher_key = None

    async def conectar(self, host, port):
        reader, writer = await asyncio.open_connection(host, port)

        # Send public key to receiver
        writer.write(self.crypto.serializarChavePublica(self.chavePublica))
        await writer.drain()

        # Receive receiver's public key
        receiverBytesChavePublica = await reader.readline()
        self.receiver_public_key = self.crypto.deserializarChavePublica(receiverBytesChavePublica)

        # Generate shared key
        chavePartilhada = self.chavePrivada.exchange(ec.ECDH(), self.receiver_public_key)

        # Derive cipher and mac keys
        salt = self.crypto.gerarNounce()
        info = b'cipher key'
        cipher_key_material = self.crypto.derivadaHKDF(chavePartilhada, salt, info=info)
        info = b'mac key'
        mac_key_material = self.crypto.derivadaHKDF(chavePartilhada, salt, info=info)
        self.cipher_key = cipher_key_material[:16]
        self.mac_key = mac_key_material[:16]

        # Send salt to receiver
        writer.write(salt)
        await writer.drain()

        return reader, writer

    async def enviarMensagem(self, mensagem, writer):
        # Generate nonce
        nonce = self.crypto.gerarNounce()

        # Encrypt message
        textoCifrado = self.crypto.cifrar(mensagem.encode(), self.cipher_key, nonce)

        # Generate hmac
        dadosAssociados = json.dumps({'nonce': nonce.decode()}).encode()
        digest = self.crypto.hmac_digest(dadosAssociados + textoCifrado, self.mac_key)

        # Send message and nonce to receiver
        data = {'cipher_text': textoCifrado.decode(), 'nonce': nonce.decode(), 'digest': digest.decode()}
        writer.write(json.dumps(data).encode())
        await writer.drain()

In [5]:
class Receiver:
    def __init__(self):
        self.crypto = funcoesCifragem()
        self.chavePrivada, self.chavePublica = self.crypto.gerarParChaves()
        self.cipher_key = None
        self.mac_key = None

    async def handlerConexao(self, reader, writer):
        # Receive emitter's public key
        emitterBytesChavePublica = await reader.readline()
        self.emitterChavePublica = self.crypto.deserializarChavePublica(emitterBytesChavePublica)

        # Send public key to emitter
        writer.write(self.crypto.serializarChavePublica(self.chavePublica))
        await writer.drain()

        # Generate shared key
        chavePartilhada = self.chavePrivada.exchange(ec.ECDH(), self.emitterChavePublica)

        # Receive salt from emitter
        salt = await reader.read(self.crypto.digest_size)

        # Derive cipher and mac keys
        info = b'cipher key'
        cipher_key_material = self.crypto.derivadaHKDF(chavePartilhada, salt, info=info)
        info = b'mac key'
        mac_key_material = self.crypto.derivadaHKDF(chavePartilhada, salt, info=info)
        self.cipher_key = cipher_key_material[:16]
        self.mac_key = mac_key_material[:16]

        # Receive messages from emitter
        while True:
            data = await reader.readline()
            if not data:
                break

            # Parse data
            data = json.loads(data)

            # Verify hmac
            nonce = data['nonce'].encode()
            textoCifrado = data['cipher_text'].encode()
            digest = data['digest'].encode()
            dadosAssociados = json.dumps({'nonce': data['nonce']}).encode()
            if not self.crypto.verificarHMAC(dadosAssociados + textoCifrado, self.mac_key, digest):
                print('Invalid message received')
                continue

            # Decrypt message
            plain_text = self.crypto.decifrar(textoCifrado, self.cipher_key, nonce)
            mensagem = plain_text.decode()

            # Process message
            print('Received message:', mensagem)

In [6]:
async def aysnc_main():
    # Create emitter and receiver
    receiver = Receiver()
    emitter = Emitter(receiverBytesChavePublica=receiver.crypto.serializarChavePublica(receiver.chavePublica))

    # Start server and connect to it
    server = await asyncio.start_server(receiver.handlerConexao, host='127.0.0.1', port=8888)
    reader, writer = await emitter.conectar('127.0.0.1', 8888)

    # Send messages
    await emitter.enviarMensagem('Hello, Receiver!', writer)
    await emitter.enviarMensagem('How are you?', writer)

    # Close connection
    writer.close()
    await writer.wait_closed()
    server.close()
    await server.wait_closed()

In [7]:
async def main():
    await async_main()

if __name__ == '__main__':
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop